# multinomialNB()参数输入的要求是什么？为什么会报错，明天看一下

In [87]:
import sklearn
import nltk

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import os
import time
import random
import jieba
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt

In [76]:
#词统计
def make_word_set(word_file):
    words_set=set()
    with open(word_file,'r',encoding='utf8') as fp:
        for line in fp.readlines():
            word=line.strip()
            if len(word)>0 and word not in words_set:
                words_set.add(word)
    return words_set

In [88]:
#文本 处理，也就是样本生成过程
def text_processing(folder_path,test_size=0.2):
    folder_list=os.listdir(folder_path)#os.listdir:返回指定路径下的文件和文件夹列表
    data_list=[]
    class_list=[]
    
    #遍历文件夹
    for folder in folder_list:
        new_folder_path=os.path.join(folder_path,folder)#os.path.join:把目录和文件名合成一个路径
        files=os.listdir(new_folder_path)
        #读取文件
        j=1
        for file in files:
            if j>100:#怕内存爆掉，只取50个样本文件
                break
            with open(os.path.join(new_folder_path,file),'r',encoding='utf') as fp:
                raw=fp.read()
            #jieba分词
            word_cut=jieba.cut(raw,cut_all=False)#精确模式，返回一个课generate对象
            word_list=list(word_cut)#化为list对象，注意编码格式为unicom
            
            data_list.append(word_list)#训练集list
            class_list.append(folder)#类别
            j+=1
    
    #粗暴的划分训练集与测试集合
#     data_class_list=zip(data_list,class_list)
#     data_class_list=list(data_class_list)
# #     print(data_class_list)
#     random.shuffle(data_class_list)
#     index=int(len(data_class_list)*test_size)+1
#     train_list=data_class_list[index:]
#     test_list=data_class_list[:index]
#     train_data_list,train_class_list=list(zip(*train_list))
#     test_data_list,test_class_list=list(zip(*test_list))
    
    train_data_list, test_data_list, train_class_list, test_class_list = sklearn.cross_validation.train_test_split(data_list, class_list, test_size=test_size)
    
    #统计词频放入all_words_dict
    all_words_dict={}
    for word_list in train_data_list:
        for word in word_list:
            if word in all_words_dict:
                all_words_dict[word]+=1
            else:
                all_words_dict[word]=1
    
    #key函数利用词频进行降序排序
    all_words_tuple_list=sorted(all_words_dict.items(),
                               key=lambda f:f[1],reverse=True)# 内建函数sorted参数需为list
    all_words_list=list(zip(*all_words_tuple_list))[0]
    print(len(all_words_list),len(train_data_list),
          len(train_class_list),len(test_data_list),len(test_class_list))
    return all_words_list,train_data_list,train_class_list,test_data_list,test_class_list


    

In [89]:
def words_dict(all_words_list,deleteN,stopwords_set=set()):
    #选取特征词
    feature_words=[]
    n=1
    for t in range(deleteN,len(all_words_list),1):
        if n>1000:#feature_words的维度为50
            break
        if not all_words_list[t].isdigit() and all_words_list[t] not in stopwords_set and 1<len(all_words_list[t])<5:
            feature_words.append(all_words_list[t])
            n+=1
    print(len(feature_words))
    return feature_words

In [90]:
#文本特征
def text_features(train_data_list,test_data_list,feature_words,flag='sklearn'):
    def text_features(text,feature_words):
        text_words=set(text)
        if flag=='nltk':
            #nitk特征dict
            features={word:1 if word in text_words
                     else 0 for word in feature_words}
            
        elif flag=='sklearn':
            #sklearn特征list
            features=[1 if word in text_words else 0
                     for word in feature_words]            
        else:
            features=[] 
        return features
    
    train_feature_list=[text_features(text,feature_words)
                       for text in train_data_list]
    test_feature_list=[text_features(text,feature_words)
                      for text in test_data_list]
    print((train_feature_list).shape)
    return train_feature_list,test_feature_list

In [91]:
#构建朴素贝叶斯模型
def text_classifier(train_feature_list,test_feature_list,
                   train_class_list,test_class_list,flag='nltk'):
    if flag=='nltk':
        #使用nltk分类器
        train_flist=zip(train_feature_list,train_class_list)
        test_flist=zip(test_feature_list,test_class_list)
        classifier=nltk.classify.NaiveBayesClassifier.train(train_flist)
        test_accuracy=nltk.classify.accuracy(classifier,test_flist)
    elif flag=='sklearn':
        classifier=MultinomialNB().fit(train_feature_list,train_class_list)#multinomialNB()参数输入的要求是什么？为什么会报错，明天
        test_accuracy=classifier.scocre(test_feature_list,test_class_list)
    else:
        test_accuracy=[]
#     print(test_accuracy)
    return test_accuracy

In [92]:
print ('start')
#文本预处理
folder_path='./Database/SogouC/Sample'
all_words_list,train_data_list,test_data_list,train_class_list,test_class_list=text_processing(folder_path,
                                                                                                test_size=0.2)
#生产stopwords_set
stopwords_file='./stopwords_cn.txt'
stopwords_set=make_word_set(stopwords_file)

#文本特征提取与分类
flag='sklearn'
deleteNs=range(0,1000,20)
test_accuracy_list=[]
for deleteN in deleteNs:
    feature_words=words_dict(all_words_list,deleteN,stopwords_set)
    train_feature_list,test_feature_list=text_features(train_data_list,
                                                     test_data_list,feature_words,flag)
    test_accuracy=text_classifier(train_feature_list,test_feature_list,train_class_list,test_class_list,flag)
    test_accuracy_list.append(test_accuracy)
print(test_accuracy_list)

plt.plot(deleteNs,test_accuracy_list)
plt.title('relationship of deleteNs and test_accuracy')
plt.xlabel('deleteNs')
plt.ylabel('test_accuracy')
plt.show()
print('finish')


start


AttributeError: module 'sklearn' has no attribute 'cross_validation'

In [60]:
import numpy as np
x=(1,2,3)
y=[3,2,1]


AttributeError: 'list' object has no attribute 'dtype'